In [1]:
import findspark
findspark.init('/home/hadoop/spark-2.2.2-bin-hadoop2.7')
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
conf = SparkConf().setMaster("spark://master:7077")
sc = SparkContext.getOrCreate()
sqlcontext = SQLContext(sc)
import pyspark.sql.functions as F

In [2]:
token_count=sqlcontext.read.parquet('hdfs://master:9000/user/hadoop/token_count')

In [5]:
wv_df = sqlcontext.read.parquet('hdfs://master:9000/user/hadoop/w2v.parquet')
wv_df.show(5)

+-----+--------------------+
|token|              vector|
+-----+--------------------+
|  정주행|[0.17401356, 0.20...|
|  하시는|[0.027278962, 0.0...|
|   분들|[0.099800214, 0.0...|
|   추천|[0.111774914, 0.0...|
|   한번|[0.05483717, -0.0...|
+-----+--------------------+
only showing top 5 rows



In [6]:
from pyspark.ml.linalg import Vectors, VectorUDT

vectorize = F.udf(lambda w, l: w*Vectors.dense(l), VectorUDT())
joined=token_count.join(wv_df, 'token')
weighted=joined.withColumn('weighted', vectorize(joined['count'],joined['vector'])).select('token','title_id','weighted')

In [7]:
weighted.write.parquet("hdfs://master:9000/user/hadoop/weighted_vector")